In [103]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
import os
import json
from bigtree import dict_to_tree
from bigtree import tree_to_dot


In [16]:
path = 'C:/Users/18565/Desktop/Classes/VA/Project/Code/VA-Project'
file_ = open(os.path.join(path,'data/json_data.json'))
data_dic = json.loads(file_.read())

In [143]:
def create_tree(data,ignore_car_id=None,get_analomy=None):
    tree = {"start":{"value":4,'time':0}}
    for car in data:
        if ignore_car_id:
            if car['car-id'] in ignore_car_id:
                # print("here")
                continue
        if get_analomy and ignore_car_id==None:
            if car['car-id'] not in get_analomy:
                continue
        route = car['path_taken'][0]
        key = "start/"+list(route.keys())[0].split(" ")[0]
        if key in tree.keys():
            tree[key]['value']+=1
        else:
            tree[key]={"value":1,'time':0}
        
        for route in car['path_taken']:
            temp =list(route.keys())[0].split(" ")[-1]
            key+="/"
            key+= temp
            if key in tree.keys():
                tree[key]['value']+=1
                tree[key]['time']+=list(route.values())[0]
            else:
                tree[key]={"value":1,'time':list(route.values())[0]}
    return tree

# Routes Analysis of car type 1

In [100]:
root_dic = create_tree(data_dic['data']['1'])

In [102]:
root = dict_to_tree(root_dic)

In [104]:
graph = tree_to_dot(root)

In [105]:
graph.write_png(os.path.join(path,"data/fig/cartype1.png"))

We noticed 3 long routes in the Tree figure of Car type 1. Analyzing more on that

In [124]:
def get_max_length_routes(data,num_routes):
    max_lengths = [0]*num_routes
    car_ids = [0]*num_routes
    for car in data:
        for i in range(num_routes):
            if max_lengths[i] < len(car['path_taken']):
                max_lengths[i] = len(car['path_taken'])
                car_ids[i] = car['car-id']
                break
    return car_ids,max_lengths

In [125]:
max_length_route_cars_ids, max_length = get_max_length_routes(data_dic['data']["1"],3)

In [127]:
max_length_route_cars_ids,max_length

(['20154112014114-381', '20155705025759-63', '20162904122951-717'],
 [97, 69, 35])

In [130]:
df = pd.read_csv(os.path.join(path,"data/Lekagul Sensor Data.csv"))

In [131]:
for car_id in max_length_route_cars_ids:
    car_df = df[df['car-id']==car_id]
    in_time = car_df['Timestamp'].iloc[0]
    out_time = car_df['Timestamp'].iloc[-1]
    car_df.to_csv(os.path.join(path,"data/anolomy/stayed_to_long/catype1_entries_"+str(len(car_df))+"_"+car_id+".csv"))
    print((car_id,in_time,out_time,len(car_df)))
    
   

('20154112014114-381', '2015-06-12 13:41:14', '2015-07-26 23:00:51', 98)
('20155705025759-63', '2015-06-05 14:57:59', '2016-05-20 14:43:07', 70)
('20162904122951-717', '2016-03-04 12:29:51', '2016-04-10 22:14:22', 36)


In [145]:
tree_without_analomy = create_tree(data_dic['data']['1'],max_length_route_cars_ids)

In [141]:
root = dict_to_tree(tree_without_analomy)
graph = tree_to_dot(root)

In [142]:
graph.write_png(os.path.join(path,"data/fig/cartype1_without_analomy.png"))

In [146]:
tree_analomy = create_tree(data_dic['data']['1'],None,max_length_route_cars_ids)

In [149]:
root = dict_to_tree(tree_analomy)
graph = tree_to_dot(root)

In [151]:
graph.write_png(os.path.join(path,"data/fig/cartype1_analomy.png"))

In [16]:
tree ={}

In [ ]:
def check_if_already_exists(node1,node2, data):
    
    key = list(data.keys())[0]
    if node1 == key:
        return data
    if 

In [28]:
for car in data_dic['data']['1']:
    if list(car['path_taken'][0].keys())[0].split(" ")[0] == 'entrance2':
        for route in car['path_taken']:
            temp = list(route.keys())[0].split(" ")[0]
            temp_dic = tree
            while temp in 
            break
    break
    

'entrance2'